# Try Langchain with Baidu

## 1. Integrate Baidu API to Langchain

The following is the effort to integrate Baidu API to Langchain.

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_community.llms import QianfanLLMEndpoint


llm = QianfanLLMEndpoint(streaming=True)
res = llm("你好！")
print(res)

/home/za/gpts/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


你好！有什么我可以帮助你的吗？
